In [97]:
import requests
import json
# Date format MM/DD/YYYY
# Site DK=20 FD=2
DATE = "08/05/2019"
SITE = "20"
# Vars -- DATE, SITE
# This one can be used to get the player salaries
SLATES = "https://resultsdb-api.rotogrinders.com/api/slates?start=DATE&site=SITE"
# Vars -- SLATEID
CONTEST_OWNERSHIP = "https://resultsdb-api.rotogrinders.com/api/contest-ownership?_slateId=SLATEID"
# Vars -- CONTESTID, INDEXNUM
CONTEST_LINEUPS = "https://resultsdb-api.rotogrinders.com/api/entries?_contestId=CONTESTID&sortBy=points&order=desc&index=INDEXNUM&maxFinish=58750&players=&users=false&username=&isLive=false&minPoints=&maxSalaryUsed=&incomplete=false&positionsRemaining=&requiredStartingPositions="
# Vars SLATEID
CONTEST_SLATE = "https://resultsdb-api.rotogrinders.com/api/contests?slates=SLATEID&lean=true"
# Vars -- SLATEID (get summary)
SUMMARY = ""

In [98]:
"""
    pulls the different slates based on the date and the site
    date- date of the event 
    site- defaults to Draftkings=20, Fanduel=2
    sport- filters the slate by sport, defaults to None
    slate_type- fitlers on the type of contest, Classic by default
"""
sport_mapping = {"football": 1, "baseball": 2, "basketball": 3}
def get_slates(date, site="20", sport=None, slate_type="Classic"):
    slate = SLATES.replace("DATE", date).replace("SITE", site)
    slate_data = requests.get(slate).json()
    if sport is not None:
        slate_data = [slate for slate in slate_data if slate['sport'] == sport_mapping[sport]]
    if slate_type is not None:
        slate_data = [slate for slate in slate_data if slate['slateTypeName'] == slate_type]
    return slate_data

In [99]:
"""
    Fetches the ownerships for a particular slate. Returns the data as a dictionary of players 
    with individual contest ownership after
    slate_id - id of the slate found with the get_slates function
    returns daily_ownerships dict
"""
def get_contest_ownerships(slate_id):
    contest_ownership_url = CONTEST_OWNERSHIP.replace("SLATEID", slate_id)
    daily_ownerships = requests.get(contest_ownership_url).json()
    return daily_ownerships

In [100]:
"""
    Fetches the contests from the slate on that day.
    slate_id -- id of the slate to find contests for
    returns contests in that slate
"""
def get_contests_from_slate(slate_id):
    contests_in_slate_url = CONTEST_SLATE.replace("SLATEID", slate_id)
    contests_in_slate = requests.get(contests_in_slate_url).json()
    return contests_in_slate

In [101]:
"""
    Fetches the lineups 25 at a time based on the contest_id and index passed
    contest_id- id of the contest to fetch (found in the summary call)
    index- index of the lineups to fetch (fetches number 25*(index) - 25*(index+1))
    returns lineups
"""
def get_contest_lineups(contest_id, index=0):
    contest_lineup_url = CONTEST_LINEUPS.replace("CONTESTID", contest_id)
    daily_lineups = requests.get(contest_lineup_url).json()
    return daily_lineups

In [102]:
daily_slate = get_slates(DATE, SITE, sport="baseball")
SLATEID = daily_slate[1]['_id']
daily_contest_lineups = get_contest_lineups(CONTEST_ID)
contests_in_slate = get_contests_from_slate(SLATEID)
contests_in_slate[0].keys()
CONTEST_ID = "5d7f7a67d0585a501d5c53be"
len(daily_ownerships)
daily_ownerships.keys()

dict_keys(['Max Scherzer', 'Lance Lynn', 'Stephen Strasburg', 'Chris Sale', 'Mike Clevinger', 'Trevor Bauer', 'Blake Snell', 'Lucas Giolito', 'Shane Bieber', 'Walker Buehler', 'Patrick Corbin', 'Charlie Morton', 'Jose Berrios', 'Clayton Kershaw', 'Tyler Glasnow', 'Matthew Boyd', 'Robbie Ray', 'Rich Hill', 'Frankie Montas', 'Luis Castillo', 'Aaron Nola', 'Luis Severino', 'Brandon Woodruff', 'Mike Minor', 'Pedro Payano', 'Domingo German', 'Carlos Martinez', 'Andrew Heaney', 'Carlos Rodon', 'Jameson Taillon', 'Jesus Luzardo', 'Corey Kluber', 'Eduardo Rodriguez', 'Kyle Hendricks', 'Michael Pineda', 'Cole Hamels', 'Jose Suarez', 'Freddy Peralta', 'Luiz Gohara', 'Alex Reyes', 'Robert Stephenson', 'Mike Soroka', 'David Price', 'Richard Lovelady', 'Yu Darvish', 'Carlos Carrasco', 'Cam Bedrosian', 'Michael Fulmer', 'Andrew Triggs', 'Masahiro Tanaka', 'Kyle Gibson', 'Brent Honeywell', 'John Lamb', 'James Paxton', 'Taylor Widener', 'Jeff Samardzija', 'Johnny Cueto', 'Yonny Chirinos', 'Chris Bassi

In [104]:
daily_slate[0]

{'_id': '5d496dede7d5973f2bdf7d32',
 'isSalary': True,
 'slateTypeName': 'Classic',
 'gameCount': 3,
 'siteSlateId': '28766',
 'end': '2019-08-06T02:10:00.000Z',
 'start': '2019-08-06T01:40:00.000Z',
 'sport': 2,
 'siteId': 20,
 'slateType': 2,
 'gameTypeId': 2,
 'updatedAt': '2019-08-06T12:09:57.112Z',
 '__v': 2,
 'startingPositions': ['P',
  'P',
  'C',
  '1B',
  '2B',
  '3B',
  'SS',
  'OF',
  'OF',
  'OF'],
 'complete': True,
 'optimalLineup': [{'name': 'Tony Gonsolin',
   'id': '13082497',
   'salary': 7000,
   'fpts': 29.7,
   'per_point': 235.69,
   'position': 'P',
   'ownership': '18.81%'},
  {'name': 'Erick Fedde',
   'id': '13082342',
   'salary': 5600,
   'fpts': 17.9,
   'per_point': 312.85,
   'position': 'P',
   'ownership': '20.79%'},
  {'name': 'J.T. Realmuto',
   'id': '13082260',
   'salary': 4400,
   'fpts': 16,
   'per_point': 275,
   'position': 'C',
   'ownership': '33.66%'},
  {'name': 'Joc Pederson',
   'id': '13082268',
   'salary': 4000,
   'fpts': 21,
   'pe